# LangChain & Azure OpenAI Notebook

# Table of Contents

### Installation & Setup
- `%pip install langchain langchain-community openai`  
- `%pip install -q pdf2image pdfminer pypdf`  
- Google Colab secrets for Azure OpenAI keys

### Connect Azure OpenAI LLM
- Initialize `AzureChatOpenAI` with endpoint, key, deployment, version, temperature

### Prompt Templates
- `PromptTemplate` and `ChatPromptTemplate`  
- Placeholders `{problem}`, `{language}`, `{text}`  
- `format()` and `format_prompt()`

### LLMChain
- Combine PromptTemplate with LLM  
- `verbose=True` for debugging  
- `run()` vs `invoke()`  

### Pipelines with Output Parsers
- `StrOutputParser` to extract plain text  
- Use `|` operator to chain prompt → LLM → parser  

### Sequential Chains
- `SimpleSequentialChain` to chain multiple LLMChains sequentially  

### Interactive Chat Loops
- Basic input/output loop with `input()`  
- Exit conditions: `exit`, `quit`, `bye`  
- Conversation history and memory  

### Conversation Memory
- `ConversationBufferMemory` for multi-turn context  
- `FileChatMessageHistory` for persistent memory  

### Caching
- `InMemoryCache` for temporary caching  
- `SQLiteCache` for persistent caching  

### Token Management
- `llm.get_num_tokens()` for input, output, and total tokens  
- Function to display output and token usage  

###  Summarization Chains
- `load_summarize_chain`  
- Chain types: `stuff`, `map_reduce`, `refine`  
- Handling large documents with `RecursiveCharacterTextSplitter`  

###  Custom Summarization Prompts
- `initial_prompt` → first concise summary  
- `refine_prompt` → refine summary with introduction, bullet points, conclusion  

### PDF Document Handling
- `PyPDFLoader` to load PDFs  
- Upload PDFs in Colab and automatically detect filenames  
- Convert PDF pages into `Document` objects  

### Chunking Large Text
- `RecursiveCharacterTextSplitter` to split text/PDFs into chunks  
- `chunk_size`, `chunk_overlap` for managing large inputs  

### Map-Reduce & Refine Summarization
- `map_reduce` → summarize each chunk then combine  
- `refine` → iteratively improve summary for large inputs  
- Display final refined summary


### Install Required Libraries
This section installs and upgrades the essential libraries used in this notebook:

- **LangChain** → A framework for building applications powered by large language models (LLMs).  
- **LangChain Community** → Provides integrations and community-driven modules for LangChain.  
- **OpenAI** → The official client library for accessing OpenAI’s GPT models.

> **Note:**
> Using `%pip` instead of `!pip` ensures that newly installed packages are **immediately available** in the current Colab kernel without restarting.

In [ ]:
# Install or upgrade all necessary packages quietly
# %pip automatically updates the current Python environment in Google Colab
# --upgrade ensures you get the latest versions of each package
# -q (quiet mode) hides unnecessary installation logs for a cleaner notebook output

%pip install --upgrade -q langchain langchain-community openai

#!pip install --upgrade -q langchain langchain-community openai

### Connect to Azure OpenAI in Google Colab
This section retrieves your **Azure OpenAI endpoint** securely from Colab’s `userdata`.  
Instead of hard-coding credentials, we use `userdata.get()` to fetch values stored under **Colab → Settings → Secrets**.  

> **Note:**  
> Always store sensitive data (like API keys or endpoints) in `userdata` for security — never type them directly into the notebook.


In [ ]:
# Import the userdata module from Google Colab
# This allows you to access securely stored secrets (like API keys or endpoints)
from google.colab import userdata

# Retrieve the Azure OpenAI endpoint from Colab's secret storage
# Replace 'AZURE_OPENAI_ENDPOINT' with your variable name used when saving the secret
AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

# Print the endpoint to confirm it's loaded (optional)
# Avoid printing actual API keys in shared environments
print(AZURE_OPENAI_ENDPOINT)


https://ai-new-agent-resource.openai.azure.com/


### Access Azure OpenAI API Key Securely
This section retrieves your **Azure OpenAI API key** from Google Colab’s `userdata` store.  
By using `userdata.get()`, you can safely load keys or tokens saved in **Colab → Settings → Secrets**,  
keeping your credentials secure and out of the public notebook.  

> **Best Practice:**  
> Never hard-code API keys directly into your code. Use `userdata` or environment variables instead.


In [ ]:
# Import the userdata module from Google Colab
# This lets you securely access any stored secrets like API keys or endpoints
from google.colab import userdata

# Retrieve the Azure OpenAI API key from Colab’s secret storage
# Ensure you've saved it under the name 'AZURE_OPENAI_KEY' in Colab > Settings > Secrets
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')

# Print the key to verify it was retrieved successfully
# Avoid printing in shared environments — this is for testing only
print(AZURE_OPENAI_KEY)


3KZOVIIWVSSadEQyVLMr722pVoXVOl6BvatHtqrAsnmSV06FL1SCJQQJ99BIACHYHv6XJ3w3AAAAACOGGKR1


### Retrieve Azure OpenAI Deployment Name
This section loads your **Azure OpenAI deployment name** from Google Colab’s `userdata` secrets.  

The deployment name identifies which specific model (e.g., *gpt-4*, *gpt-4o-mini*) you’ve configured inside your Azure OpenAI resource.

> **Tip:**  
> You can store it securely under **Colab → Settings → Secrets** with the key name `DEPLOYMENT_NAME`.



In [ ]:
# Import the userdata module from Google Colab
# This allows secure access to values like deployment names, API keys, and endpoints
from google.colab import userdata

# Retrieve the Azure OpenAI deployment name
# Make sure you've saved it under 'DEPLOYMENT_NAME' in Colab → Settings → Secrets
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')

# Print the deployment name to verify it was loaded successfully
# Avoid exposing sensitive details when sharing notebooks
print(DEPLOYMENT_NAME)


gpt-4o


### Integrate LangChain with Azure OpenAI
This section configures LangChain to use your **Azure OpenAI deployment** as the language model backend.  
LangChain provides a high-level interface for building LLM-powered apps like chatbots, agents, and workflows.  

> **Parameters Explanation:**  
> - `openai_api_base` → Your Azure OpenAI endpoint URL.  
> - `openai_api_key` → Your secret API key for authentication.  
> - `openai_api_version` → The version of the Azure OpenAI API you’re using.  
> - `deployment_name` → The name of your deployed model (e.g., *gpt-4o*, *gpt-35-turbo*).  
> - `openai_api_type="azure"` → Required flag to specify you’re connecting to Azure’s API.



This LLM connection will be used for the **entire project/class demo** for all summarization, chat, and other examples.

In a LangChain project, the only part you need to change when switching LLM providers or models is the LLM integration itself. Everything else—chains, prompts, memory, summarization logic, streaming, token counting—remains the same.

In [ ]:
# Import the AzureChatOpenAI class from LangChain
# This class lets you use Azure OpenAI models as LangChain-compatible LLMs
from langchain.chat_models import AzureChatOpenAI

# Initialize the LLM with Azure credentials and configuration
llm = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_ENDPOINT,      # The Azure OpenAI endpoint
    openai_api_key=AZURE_OPENAI_KEY,            # API key for authentication
    openai_api_version="2023-03-15-preview",    # API version (check your Azure setup)
    deployment_name=DEPLOYMENT_NAME,            # Model deployment name
    openai_api_type="azure",                    # Indicates you're using Azure OpenAI
)

# Verify connection by sending a simple prompt
response = llm.invoke("Hello from LangChain + Azure OpenAI integration!")
print(response)


/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://ai-new-agent-resource.openai.azure.com/ to https://ai-new-agent-resource.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.co

content="Hello! It sounds like you're working with LangChain and Azure OpenAI—what an exciting integration! LangChain is fantastic for building powerful language-based applications, and combining it with Azure OpenAI makes for a robust setup. Let me know if there's anything specific you'd like help with regarding this integration!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 17, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'finish_reason': 'stop', 'logprobs': None} id='run--bcfb1184-defe-4e69-ad62-55372f8e3dfb-0'


### Create a Prompt Template and Use LLMChain
Now that the `llm` (Azure OpenAI model) is connected,  
we can build a **LangChain pipeline** using `PromptTemplate` and `LLMChain`.  

- **PromptTemplate:** Defines the structure of your input prompt with placeholders for variables.  
- **LLMChain:** Combines the prompt and the model to process inputs and generate responses automatically.

In [ ]:
# Import LangChain components for prompt creation and chaining
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define a reusable prompt template with a variable placeholder {topic}
prompt = PromptTemplate(
    input_variables=["topic"],  # Variable to be replaced at runtime
    template="Explain the concept of {topic} in simple terms."  # The prompt text
)

# Create an LLMChain that combines the template and the connected Azure LLM
chain = LLMChain(
    llm=llm,             # Azure OpenAI model configured earlier
    prompt=prompt,       # Prompt template
    verbose=True         # Displays intermediate steps and outputs
)

# Run the chain by providing a value for the {topic} variable
response = chain.run({"topic": "Edge and Fog Computing"})

# Display the model's response
print(response)




> Entering new LLMChain chain...
Prompt after formatting:
Explain the concept of Edge and Fog Computing in simple terms.

> Finished chain.
Certainly! Edge Computing and Fog Computing are concepts related to how and where data is processed in a network, especially in systems like the Internet of Things (IoT). Let me break them down simply:

### **Edge Computing**
- **What it is**: Edge Computing refers to processing data close to where it is created (at the "edge" of the network). Instead of sending all the data to a central server or cloud for processing, devices like sensors, cameras, or small local servers handle the data nearby.
- **Why it's useful**: It reduces the time (latency) it takes to process data and minimizes the amount of data sent to the cloud, saving bandwidth and improving speed.
- **Example**: Imagine a security camera analyzing video footage locally to detect motion instead of sending all footage to the cloud for analysis.

---

### **Fog Computing**
- **What it i

### Build an Interactive Chat Loop
This section creates a basic **chat interface** in the notebook terminal using Python’s `input()` function.  
It allows users to type a message, send it to the LangChain `LLMChain`, and get a response back.  

> 💡 **Tip:**  
> Type `exit` anytime to stop the chat loop.  
> This is a simple example for local or Colab testing —  
> for web apps, you can later integrate it with **Flask** or **Streamlit**.


In [ ]:
# Start an infinite loop to keep the conversation going
while True:
    # Prompt the user for input
    user_input = input("You: ")

    # Exit condition — breaks the loop if user types 'exit'
    if user_input.lower() == "exit":
        print("Chat session ended.")
        break

    # Pass the user's input to the LangChain pipeline (LLMChain)
    response = chain.run(user_input)

    # Display the assistant's response
    print("Assistant:", response)


You: Edge and Fog Computing


> Entering new LLMChain chain...
Prompt after formatting:
Explain the concept of Edge and Fog Computing in simple terms.

> Finished chain.
Assistant: Certainly! Here's a simple explanation of **Edge Computing** and **Fog Computing**:

---

### **Edge Computing:**
Think of edge computing as moving data processing closer to where the data is generated or used — at the "edge" of the network. Instead of sending all the data to a central server or cloud, devices like sensors, cameras, or local machines process the data right where it’s created. This reduces the time it takes to respond (lower latency) and decreases the load on central systems.

Example: Imagine a smart security camera that can identify faces. Instead of sending video footage to a distant cloud server for analysis, the camera itself processes the video locally and identifies faces. This makes the process faster and more efficient.

---

### **Fog Computing:**
Fog computing is a layer between ed

### Using Structured Messages with LangChain
LangChain supports structured message types — similar to how chat models like ChatGPT work:  
- **SystemMessage:** Defines the assistant’s behavior or personality.  
- **HumanMessage:** Represents user input.  
- **AIMessage:** Represents the AI’s response (optional for initialization).

This approach helps when building **multi-turn chat applications**, where messages are passed in a structured list instead of plain text.


In [ ]:
# Import message schema classes from LangChain
# These are used to structure system, user, and AI messages in a conversation
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# Create a list of messages to simulate a chat conversation
messages = [
    SystemMessage(content="You are a helpful assistant."),      # Defines AI role
    HumanMessage(content="What is the capital of France?")       # User query
]

# Print the list to show its structure
print(messages)

# Send the structured messages to the Azure LLM (connected earlier)
Output = llm.invoke(messages)

#  Print the full response object
print(Output)

# Extract and print only the AI-generated message content
print(Output.content)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]
content='The capital of France is **Paris**.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--7cf89e0e-8491-46fc-b0fe-3da128670518-0'
The capital of France is **Paris**.


### Measure LLM Execution Time (No Cache)
This cell sends a prompt to the Azure OpenAI model and measures how long it takes **without any caching**.  
Useful for understanding the baseline latency of your model request.


In [ ]:
# %%time measures the execution time of the cell
%%time

#  Define a prompt to send to the LLM
prompt = "Tell me a joke"

# Invoke the LLM and get the response
response = llm.invoke(prompt)

# Print the response from the model
print(response)


content="Sure! Here's a lighthearted one for you:\n\nWhy don't skeletons fight each other?\n\nBecause they don’t have the guts! 😄" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 11, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'finish_reason': 'stop', 'logprobs': None} id='run--ba3e9303-10cf-4218-b7f3-e5f4218ac576-0'
CPU times: user 19.4 ms, sys: 0 ns, total: 19.4 ms
Wall time: 504 ms


### Enable In-Memory Caching for LangChain
This cell sets up caching so repeated calls with the same prompt **reuse previous results**.  
In-memory caching improves performance and reduces API calls to Azure OpenAI.  
We use **LangChain’s InMemoryCache** and `set_llm_cache()` for this purpose.


In [ ]:
# Import caching utilities from LangChain
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# Set up an in-memory cache for the LLM
# This will store responses and reuse them for identical prompts
set_llm_cache(InMemoryCache())


### ⏱ Measure LLM Execution Time (With Cache)
Now we run the **same prompt** again with caching enabled.  
The execution time should be much faster because the response is retrieved from memory.


In [ ]:
# %%time measures how long the cell takes to execute
%%time

# Same prompt as before
prompt = "Tell me a joke"

# Invoke the LLM; cached response will be used if available
response = llm.invoke(prompt)

# Print the response
print(response)


content="Sure! Here's one for you:\n\nWhy don't skeletons fight each other?\n\nBecause they don't have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 11, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'finish_reason': 'stop', 'logprobs': None} id='run--756c5464-05a5-4676-8422-94c224d8241a-0'
CPU times: user 6.05 ms, sys: 144 µs, total: 6.2 ms
Wall time: 440 ms


### Measure LLM Execution Time
This cell demonstrates how to measure the execution time of a single LLM call using the `%%time` in Colab.  
It helps understand how long the Azure OpenAI model takes to process a prompt.


In [ ]:
# %%time is a Jupyter/Colab magic command
# It measures the total time taken by this cell to run
%%time

# Define a prompt to send to the LLM
prompt = "Tell me a joke"

# Invoke the LLM and get the response
response = llm.invoke(prompt)

# Print the response returned by the model
print(response)


content="Sure! Here's one for you:\n\nWhy don't skeletons fight each other?\n\nBecause they don't have the guts!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 11, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'finish_reason': 'stop', 'logprobs': None} id='run--756c5464-05a5-4676-8422-94c224d8241a-0' usage_metadata={'total_cost': 0}
CPU times: user 1.14 ms, sys: 19 µs, total: 1.16 ms
Wall time: 1.15 ms


### Enable SQLite Caching for LangChain
This cell configures **SQLite-based caching** for your LangChain LLM.  
Using SQLite cache allows responses to be stored persistently on disk,  
so repeated prompts can retrieve results without making extra API calls, even across sessions.

> 📝 **Note:**  
> The database file `.langchain.db` will be created in the current working directory.


In [ ]:
# Import SQLiteCache from LangChain
from langchain.cache import SQLiteCache

# Set up a persistent SQLite cache for the LLM
# database_path specifies the file where cached responses will be stored
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


### ⏱ Generate Content with LLM (Using SQLite Cache)
Now that SQLite caching is enabled, repeated calls with the **same prompt** will be faster.  
The response is stored in `.langchain.db`, so the LLM can return cached results without extra API calls.

> 💡 **Tip:**  
> This is useful for cost and speed optimization when testing or iterating on prompts.

In [ ]:
# %%time measures the execution time of this cell
%%time

# Define the same creative prompt as before
prompt = "write a rock song about moon."

# Invoke the LLM; response may come from cache if the prompt was run previously
response = llm.invoke(prompt)

# Print only the generated content
print(response.content)


**"Lunar Reverie"**  
*(A Rock Anthem for the Moon)*  

---

**[Verse 1]**  
Silver sentinel in the midnight sky,  
You pull the tides, you make the wolves cry.  
Hanging high, cold and alone,  
The keeper of secrets, carved in stone.  

Whispers of shadows dance in your glow,  
A silent witness to all we’ll never know.  
Ancient queen of the endless night,  
You’re the muse of dreamers, burning bright.  

---

**[Pre-Chorus]**  
Through the darkness, your light remains,  
A beacon calling through the cosmic chains.  
Every lover’s gaze, every poet’s plea,  
You’re the rhythm of eternity.  

---

**[Chorus]**  
Oh moon, shine down on me tonight,  
Illuminate the chaos, make it feel alright.  
You’re the song in my veins, the fire in my soul,  
The lunar lover I’ll never let go.  

Oh moon, you’ve got me hypnotized,  
A rebel heart with your silver disguise.  
You’re the pulse of the night, the edge of the dream,  
Forever burning with that ghostly gleam.  

---

**[Verse 2]**  
You’ve 

### Re-run LLM Prompt with SQLite Cache
This cell demonstrates running the same prompt again after enabling **SQLite caching**.  
Since the prompt was executed previously, the response should be **retrieved from cache**, making it much faster.

> 💡 **Note:**  
> Using a persistent cache helps reduce API calls and improves response time for repeated prompts.


In [ ]:
# %%time measures the execution time of this cell
%%time

# Define the prompt (same as before)
prompt = "write a rock song about moon."

# Invoke the LLM; cached response will be returned if available
response = llm.invoke(prompt)

# Print only the generated content from the model
print(response.content)


**"Lunar Reverie"**  
*(A Rock Anthem for the Moon)*  

---

**[Verse 1]**  
Silver sentinel in the midnight sky,  
You pull the tides, you make the wolves cry.  
Hanging high, cold and alone,  
The keeper of secrets, carved in stone.  

Whispers of shadows dance in your glow,  
A silent witness to all we’ll never know.  
Ancient queen of the endless night,  
You’re the muse of dreamers, burning bright.  

---

**[Pre-Chorus]**  
Through the darkness, your light remains,  
A beacon calling through the cosmic chains.  
Every lover’s gaze, every poet’s plea,  
You’re the rhythm of eternity.  

---

**[Chorus]**  
Oh moon, shine down on me tonight,  
Illuminate the chaos, make it feel alright.  
You’re the song in my veins, the fire in my soul,  
The lunar lover I’ll never let go.  

Oh moon, you’ve got me hypnotized,  
A rebel heart with your silver disguise.  
You’re the pulse of the night, the edge of the dream,  
Forever burning with that ghostly gleam.  

---

**[Verse 2]**  
You’ve 

### Disable LangChain Cache
This cell demonstrates how to **turn off caching** in LangChain.  
Disabling cache ensures that every LLM call goes directly to the API, which is useful for testing or when you want fresh responses every time.

> 💡 **Note:**  
> After disabling cache, repeated prompts will no longer retrieve stored responses.


In [ ]:
# Import the set_llm_cache function from LangChain globals
from langchain.globals import set_llm_cache

# Disable caching by setting it to None
set_llm_cache(None)

# Confirm that cache is disabled by sending a prompt
prompt = "Write a short poem about the sun."
response = llm.invoke(prompt)
print(response.content)


Oh radiant orb, fierce and bright,  
You paint the sky with golden light.  
A tender touch, a blazing flame,  
Each dawn, you rise to stake your claim.  

Through whispered winds and ocean's roar,  
You warm the earth, our hearts, and more.  
A fleeting glow at twilight's crest,  
In your embrace, the world finds rest.  


### Generate a New Rock Song
This cell sends a prompt to the Azure OpenAI LLM to generate a **rock song about the Moon**.  
We directly print the LLM’s response content.


In [ ]:
# %%time measures total execution time for streaming
%%time

# Define a creative prompt for the LLM
prompt = "Write a rock song about the Moon."

# Invoke the LLM and print only the content
print(llm.invoke(prompt).content)


**"Lunar Calling"**  
*(A Rock Anthem)*  

**[Verse 1]**  
Silver guardian in the night,  
Hanging high in cosmic light.  
You pull the tides, you guide the seas,  
A mystery wrapped in galaxies.  
Shadows dance across your face,  
A timeless witness to endless space.  

**[Pre-Chorus]**  
I hear your whisper, I feel your glow,  
A silent rebel in the skies you show.  

**[Chorus]**  
Moon! You’re calling me, pulling me higher,  
Burning bright like a cold-fire.  
Moon! You’re haunting my soul,  
A rock renegade with full control.  

**[Verse 2]**  
Craters carved by time untold,  
Secrets hidden in your folds.  
A lonely traveler, no breath, no sound,  
Yet your presence shakes the ground.  
Every howl, every prayer,  
Echoes deep in lunar air.  

**[Pre-Chorus]**  
I hear your whisper, I feel your glow,  
A silent rebel in the skies you show.  

**[Chorus]**  
Moon! You’re calling me, pulling me higher,  
Burning bright like a cold-fire.  
Moon! You’re haunting my soul,  
A rock rene

### Stream a Long Response from the LLM
This cell demonstrates **streaming responses** from the LLM using `llm.stream()`.  
Streaming allows partial output to appear as it is generated, useful for **long responses or real-time display**.

> 💡 **Tip:**  
> `end=""` ensures that the content is printed continuously on the same line,  
> and `flush=True` forces immediate display of each chunk.


In [ ]:
# %%time measures total execution time for streaming
%%time

# Define a prompt for streaming
prompt = "write a rock new song about moon and sun."

# Stream the response from the LLM
for chunk in llm.stream(prompt):
    # Print each chunk as it is generated
    print(chunk.content, end="", flush=True)


**"Moon and Sun"**  
*(A Rock Anthem)*  

**[Verse 1]**  
Out in the night, the moon’s got her glow,  
Silver queen watching as the world moves slow.  
She pulls the tides, she whispers to the stars,  
A quiet rebel with scars that are ours.  

The sun wakes up, blazing golden fire,  
A restless king with eternal desire.  
He lights the skies, burns away the cold,  
A story of power that never gets old.  

**[Pre-Chorus]**  
They’re caught in a cosmic dance,   
Chasing shadows, chasing chance.  
Two forces locked but never collide,  
Forever apart, yet side by side.  

**[Chorus]**  
Moon and Sun, opposites in the sky,  
One brings the darkness, one lets it die.  
Locked in a rhythm, a love they can't touch,  
Moon and Sun, they mean so much.  

**[Verse 2]**  
The moon has secrets she’ll never reveal,  
A quiet beauty, cool and surreal.  
The sun’s got fury, a heart ablaze,  
Burning brighter through endless days.  

They’re both alone, yet they’re never apart,  
Two halves of a unive

### Create a Custom Prompt Template
This cell demonstrates how to define a **custom prompt template** in LangChain.  
We use placeholders (`{problem}`, `{language}`) to dynamically generate prompts based on user input.

> 💡 **Tip:**  
> Prompt templates help you standardize and reuse prompts across multiple LLM calls.


In [ ]:
# Import PromptTemplate from LangChain
from langchain.prompts import PromptTemplate

# Define a template string with placeholders
template = '''You are an experienced machine learning engineer.
Choose which type of problem this {problem} is, based on data,
and provide the desired result in {language}.'''

# Create a PromptTemplate object from the template string
prompt_template = PromptTemplate.from_template(template)

# Format the template by providing actual values for the placeholders
prompt = prompt_template.format(
    problem="you have to predict how much ice cream will sell in this festival sale",
    language="Hindi"
)

# Print the final formatted prompt
print(prompt)


You are an experienced machine learning engineer.
Choose which type of problem this you have to predict how much ice cream will sell in this festival sale is, based on data, 
and provide the desired result in Hindi.


### Invoke the LLM and Print Output
This cell demonstrates how to **send a prompt to the LLM** using `llm.invoke()`  
and extract the generated text using `output.content`.


In [ ]:
# Invoke the LLM with the previously defined prompt
output = llm.invoke(prompt)

# Print only the content of the response
print(output.content)


यह समस्या एक **Regression Problem** है, क्योंकि आपको किसी दिए गए डेटा के आधार पर एक संख्या (कितनी आइसक्रीम बिकेगी) की भविष्यवाणी करनी है। 

### हिंदी में उत्तर:
यह समस्या एक **रेग्रेशन समस्या** है।


### Create a Chat Prompt Template
This cell demonstrates how to build a **chat prompt template** using structured messages:  
- **SystemMessage** → Defines the assistant’s behavior or constraints.  
- **HumanMessagePromptTemplate** → Represents user input with placeholders.  

We can then format the prompt with actual input and convert it to a list of messages for the LLM.


In [ ]:
# Import necessary classes from LangChain
from langchain.prompts import HumanMessagePromptTemplate, ChatMessagePromptTemplate, ChatPromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with a system message and a human input placeholder
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You respond only in JSON Format."),  # Assistant behavior
        HumanMessagePromptTemplate.from_template("{user_input}")   # Placeholder for user input
    ]
)

# Format the template with actual user input
chat_prompt = chat_template.format_prompt(
    user_input="Top 5 countries in world by population."
).to_messages()  # Convert the formatted prompt to messages

# Print the structured chat messages ready to be sent to the LLM
print(chat_prompt)


[SystemMessage(content='You respond only in JSON Format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 5 countries in world by population.', additional_kwargs={}, response_metadata={})]


In [ ]:
# Invoke the LLM with the previously defined prompt
output = llm.invoke(chat_prompt)

# Print only the content of the response
print(output.content)

```json
{
  "top_countries_by_population": [
    {
      "rank": 1,
      "country": "China",
      "population": 1411778724
    },
    {
      "rank": 2,
      "country": "India",
      "population": 1406630327
    },
    {
      "rank": 3,
      "country": "United States",
      "population": 334233854
    },
    {
      "rank": 4,
      "country": "Indonesia",
      "population": 277534122
    },
    {
      "rank": 5,
      "country": "Pakistan",
      "population": 240485658
    }
  ]
}
```


### Use LLMChain with a Custom Prompt Template
This cell demonstrates how to combine a **custom prompt template** with the connected LLM using `LLMChain`.  
- **LLMChain** takes a prompt template and an LLM, then handles formatting and invoking automatically.  
- The `verbose=True` option prints intermediate steps for teaching purposes.


In [ ]:
# Import LLMChain from LangChain
from langchain.chains import LLMChain

# Define a template string with placeholders
template = '''You are an experienced machine learning engineer.
Choose which type of problem this {problem} is, based on data,
and provide the desired result in {language}.'''

# Create a PromptTemplate from the template string
prompt_template = PromptTemplate.from_template(template)

# Initialize an LLMChain with the Azure LLM and the prompt template
chain = LLMChain(
    llm=llm,             # Connected Azure OpenAI LLM
    prompt=prompt_template,
    verbose=True          # Print intermediate steps
)

# Invoke the chain with specific values for placeholders
output = chain.invoke({
    "problem": "you have to predict how much ice cream will sell in this festival sale",
    "language": "Hindi"
})

# Print the final output from the LLM
print(output)




> Entering new LLMChain chain...
Prompt after formatting:
You are an experienced machine learning engineer.
Choose which type of problem this you have to predict how much ice cream will sell in this festival sale is, based on data,
and provide the desired result in Hindi.

> Finished chain.
{'problem': 'you have to predict how much ice cream will sell in this festival sale', 'language': 'Hindi', 'text': 'यह समस्या एक **Regression Problem** है। \n\nक्योंकि यहां हमें एक निरंतर (continuous) मान की भविष्यवाणी करनी है, जैसे कि "इस त्यौहार पर कितनी आइसक्रीम बिकेगी।" Regression का उपयोग तब किया जाता है जब हमें किसी संख्या का अनुमान लगाना हो।\n\nउदाहरण के लिए, आप पिछली बिक्री, मौसम का डेटा (जैसे तापमान), त्यौहार के प्रकार, और अन्य प्रासंगिक कारकों का उपयोग कर सकते हैं ताकि अनुमान लगाया जा सके कि इस त्यौहार पर कितनी आइसक्रीम बिकेगी। \n\n### Desired result in Hindi:\nइस समस्या को हल करने के लिए Regression Algorithm का उपयोग करें, जैसे:\n1. Linear Regression\n2. Decision Tree Regression\n3. Ran

### LLM Pipeline with Output Parser
This example demonstrates how to **combine a prompt template, LLM, and an output parser** into a single pipeline using LangChain’s `|` operator:  
- `StrOutputParser()` ensures the LLM output is returned as a **plain string**.  
- This approach helps standardize outputs, especially when building applications that rely on predictable text formats.


In [ ]:
# Import StrOutputParser from LangChain core
from langchain_core.output_parsers import StrOutputParser

# Create a pipeline combining:
# Prompt template -> LLM -> String output parser
chain = prompt_template | llm | StrOutputParser()

# Invoke the chain with values for placeholders in the prompt
output = chain.invoke({
    "problem": "you have to predict how much ice cream will sell in this festival sale",
    "language": "Hindi"
})

# Print the standardized output from the pipeline
print(output)


यह समस्या एक **Regression problem** है क्योंकि इसमें आपको किसी त्योहार की बिक्री में कितनी आइसक्रीम बिकेगी, उसकी मात्रा (numerical value) का अनुमान लगाना है। 

Regression problems में हम किसी continuous variable (जैसे बिक्री की मात्रा) का अनुमान लगाते हैं। आप historical data, जैसे पिछले साल की बिक्री, मौसम, त्योहार का प्रकार, और अन्य संबंधित कारकों का उपयोग करके एक मॉडल बना सकते हैं।

### वांछित परिणाम:
यह समस्या एक Regression problem है।


### Sequential LLM Chains with SimpleSequentialChain
This example demonstrates how to **chain multiple LLMs or chains sequentially** using LangChain’s `SimpleSequentialChain`.  

- **chain_1:** Generates a Python function or explanation for a given concept.  
- **chain_2:** Further processes or refines the output, possibly using a second LLM instance.  
- `SimpleSequentialChain` executes chains in order, passing the output of one as the input to the next.

> 💡 **Tip:**  
> You can adjust `temperature` for creativity in the second LLM chain.


In [ ]:
# Import necessary classes from LangChain
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chat_models import AzureChatOpenAI

# Create the first prompt template
prompt_template_1 = PromptTemplate.from_template(
    template='You are an experienced data scientist and Python programmer. Write a function that implements the concept of {concept}.'
)

# Create the first LLMChain with the main LLM
chain_1 = LLMChain(llm=llm, prompt=prompt_template_1)

# Initialize a second AzureChatOpenAI LLM with a different temperature
llm_2 = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_KEY,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_type="azure",
    temperature=0.9
)

# Create the second prompt template
prompt_template_2 = PromptTemplate.from_template(
    template='Write a Python function that implements the concept of {concept}.'
)

# Create the second LLMChain with the second LLM
chain_2 = LLMChain(llm=llm_2, prompt=prompt_template_2)

# Combine both chains into a SimpleSequentialChain
overall = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)

# Invoke the sequential chain with a concept
output = overall.invoke("Linear Regression")

# Print the final output from the sequential chain
print(output)


/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://ai-new-agent-resource.openai.azure.com/ to https://ai-new-agent-resource.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.co



> Entering new SimpleSequentialChain chain...
Certainly! Below is a Python implementation of linear regression using the **least squares method**. This function calculates the regression line \( y = mx + b \), where \( m \) is the slope and \( b \) is the intercept.

### Implementation of Linear Regression

```python
import numpy as np

def linear_regression(X, y):
    """
    Perform linear regression to fit a line y = mx + b to the data.

    Parameters:
        X (array-like): 1D array of independent variable values (features).
        y (array-like): 1D array of dependent variable values (targets).
    
    Returns:
        dict: A dictionary containing the slope (m), intercept (b), and predicted values (y_pred).
    """
    # Ensure X and y are numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    # Calculate the means of X and y
    X_mean = np.mean(X)
    y_mean = np.mean(y)
    
    # Calculate the slope (m) using the formula: m = cov(X, y) / var(X)
    numerator = np

### Interactive Chatbot in English
This example demonstrates how to build a **real-time chatbot** in Google Colab using LangChain:  

- **AzureChatOpenAI** → The LLM instance connected to your Azure deployment.  
- **ChatPromptTemplate** → Defines the chat structure with a system message (AI behavior) and human input.  
- **StrOutputParser** → Ensures the output is returned as a plain string.  
- The `while True` loop enables continuous conversation until the user types `"exit"`.


In [ ]:
# Import necessary classes from LangChain
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

# Initialize the AzureChatOpenAI LLM
llm = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_ENDPOINT,  # Your Azure endpoint
    openai_api_key=AZURE_OPENAI_KEY,        # Your API key
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,       # Model deployment name
    openai_api_type="azure",
    temperature=0.9                        # Creativity of responses
)

# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You respond only in English."),  # System behavior
        HumanMessagePromptTemplate.from_template("{user_input}")  # Placeholder for user input
    ]
)

# Combine the prompt, LLM, and output parser into a chain
chain = prompt | llm | StrOutputParser()

# Start an interactive chat loop
while True:
    user_input = input("You: ")  # Take input from the user

    # Exit condition
    if user_input.lower() == "exit":
        print("Chat ended.")
        break

    # Invoke the chain with user input
    output = chain.invoke({"user_input": user_input})

    # Print the assistant's response
    print("Assistant:", output)
    print("_" * 50)  # Separator for readability


/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://ai-new-agent-resource.openai.azure.com/ to https://ai-new-agent-resource.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/langchain_community/chat_models/azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.co

You: hi
Assistant: Hello! How can I assist you today? 😊
__________________________________________________
You: exit
Chat ended.


### Conversation Memory with LangChain
This cell demonstrates how to **store and manage chat history** using LangChain’s `ConversationBufferMemory`.  

- **ConversationBufferMemory** → Keeps track of all previous messages in the conversation.  
- **MessagesPlaceholder** → Allows prompts to automatically include past messages when generating responses.

> 💡 **Tip:**  
> Memory is useful for multi-turn conversations where context from previous messages improves the assistant’s replies.


In [ ]:
# Import classes for memory and message placeholders
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

# Initialize a conversation memory buffer
memory = ConversationBufferMemory(
    memory_key="chat_history",  # Key used to store messages internally
    return_messages=True        # Ensures messages are returned as structured objects
)

# MessagesPlaceholder can be used in prompt templates to include chat history
# Example usage in a ChatPromptTemplate: MessagesPlaceholder(variable_name="chat_history")


### Initialize Conversation Memory
This cell sets up a **ConversationBufferMemory** object to store chat history.  

- `memory_key="chat_history"` → Internal key for storing messages.  
- `return_messages=True` → Ensures that stored messages are returned in structured format (SystemMessage, HumanMessage, AIMessage),  
  which can be reused in prompts for multi-turn conversations.


In [ ]:
# Import ConversationBufferMemory if not already imported
from langchain.memory import ConversationBufferMemory

# Initialize a memory buffer to store chat history
memory = ConversationBufferMemory(
    memory_key="chat_history",  # Key used internally for storing messages
    return_messages=True        # Return messages in structured format
)




### Add Conversation Memory to LLMChain
This cell demonstrates how to attach **ConversationBufferMemory** to an LLMChain.  

- `memory=memory` → Enables the chain to **remember previous messages**, allowing multi-turn conversations.  
- `verbose=False` → Disables intermediate debug prints; set to `True` for teaching or debugging.


In [ ]:
# Import LLMChain if not already imported
from langchain.chains import LLMChain

# Create an LLMChain with memory enabled
chain = LLMChain(
    llm=llm,        # Connected AzureChatOpenAI LLM
    prompt=prompt,  # Chat prompt template
    memory=memory,  # Conversation memory to store chat history
    verbose=False   # Set to True to see intermediate steps
)

### Interactive Multi-Turn Chat with Memory
This cell creates a **chat loop** where the assistant remembers previous messages using `ConversationBufferMemory`.  

- Users can type `"exit"`, `"quit"`, or `"bye"` to end the chat.  
- Each response is generated in the context of the conversation, allowing for **context-aware replies**.  
- `print("_" * 50)` adds a visual separator for readability.


In [ ]:
# Start an infinite chat loop
while True:
    user_input = input("You: ")  # Take user input

    # Exit conditions
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye")
        break

    # Invoke the chain with memory
    output = chain.invoke({"user_input": user_input})

    # Print the assistant's response
    print("Assistant:", output)
    print("_" * 50)  # Separator for readability


You: Hi
Assistant: {'user_input': 'Hi', 'chat_history': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={})], 'text': 'Hello! How can I assist you today?'}
__________________________________________________
You: quit
Goodbye


### 🖨 Print Plain Text Instead of JSON
This example ensures that only the **text content** from the LLM is displayed


In [ ]:
# Start interactive chat loop
while True:
    user_input = input("You: ")

    # Exit condition
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye")
        break

    # Run the chain and get plain text output
    output = chain.run({"user_input": user_input})

    # Print assistant's response
    print("Assistant:", output)
    print("_" * 50)


### Persistent Chat Memory with File Storage
This example demonstrates how to **store conversation history in a JSON file** using `FileChatMessageHistory`.  

- `FileChatMessageHistory("chat_history.json")` → Stores all messages in the specified JSON file.  
- `ConversationBufferMemory` → Wraps the file history so the LLM can access previous messages for context.  
- The chat loop continues until the user types `"exit"`, `"quit"`, or `"bye"`.


In [ ]:
# Import FileChatMessageHistory
from langchain.memory import FileChatMessageHistory
from langchain.chains import LLMChain

# Create a persistent file-based chat history
history = FileChatMessageHistory("chat_history.json")

# Initialize ConversationBufferMemory with the file history
memory = ConversationBufferMemory(
    memory_key="chat_history",  # Internal key for memory
    return_messages=True,       # Return structured messages
    chat_memory=history         # Use persistent file storage
)

# Create the LLMChain with memory enabled
chain = LLMChain(
    llm=llm,       # Connected AzureChatOpenAI LLM
    prompt=prompt, # Chat prompt template
    verbose=False, # Disable verbose logs
    memory=memory  # Attach persistent memory
)

# Start interactive chat loop with persistent memory
while True:
    user_input = input("You: ")

    # Exit conditions
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye")
        break

    # Invoke the chain with memory
    output = chain.invoke({"user_input": user_input})

    # Print assistant response
    print("Assistant:", output)
    print("_" * 50)  # Separator for readability


You: Hi
Assistant: {'user_input': 'Hi', 'chat_history': [], 'text': 'Hello! How can I assist you today?'}
__________________________________________________
You: quit
Goodbye


### 📝 Summarize a Document Using LLM
This example demonstrates how to summarize a long text using LangChain:  

- `SystemMessage` → Sets the assistant’s role and expertise.  
- `HumanMessage` → Provides the text to summarize.  
- `llm.invoke(messages)` → Sends the structured messages to the LLM and returns the response.  
- `output.content` → Extracts the plain text summary.


In [ ]:
# Import SystemMessage and HumanMessage if not already imported
from langchain.schema import SystemMessage, HumanMessage

# Define the text to summarize
text = '''
The coronavirus, officially known as COVID-19, is a highly contagious viral infection that emerged in late 2019 in the city of Wuhan, China.
Classified as a novel coronavirus, it quickly spread worldwide, leading to a global pandemic declared by the World Health Organization
(WHO) in March 2020. The virus primarily spreads through respiratory droplets and close contact, with symptoms ranging from mild, such as
fever, cough, and fatigue, to severe, including difficulty breathing and complications like pneumonia. Vulnerable populations, such as
the elderly and those with underlying health conditions, faced increased risks of severe outcomes. The pandemic had profound impacts on
global health systems, economies, and daily life, prompting widespread lockdowns, travel restrictions, and shifts to remote work and
education. In response, scientists and pharmaceutical companies rapidly developed vaccines, which became a crucial tool in combating
the virus and reducing its spread. Despite progress in managing the pandemic, the emergence of new variants posed ongoing challenges,
underscoring the importance of public health measures, vaccination campaigns, and global cooperation to mitigate the virus's impacts.
'''

# Create structured messages for the LLM
messages = [
    SystemMessage(content="You are an expert copywriter with expertise in summarizing documents."),
    HumanMessage(content=f"Please provide a short and concise summary of the following text:\n\nTEXT:\n{text}")
]

# Invoke the LLM to get the summary
output = llm.invoke(messages)

# Print the summarized content
print(output.content)


### Count Tokens for Input and Output
This example demonstrates how to compute the **number of tokens** for a given input text, the LLM’s output, and the **total tokens used**.  
> Useful for monitoring token usage and estimating costs with Azure OpenAI.


In [ ]:
# Define the input text
text = '''
The coronavirus, officially known as COVID-19, is a highly contagious viral infection that emerged in late 2019 in the city of Wuhan, China.
Classified as a novel coronavirus, it quickly spread worldwide, leading to a global pandemic declared by the World Health Organization
(WHO) in March 2020. The virus primarily spreads through respiratory droplets and close contact, with symptoms ranging from mild, such as
fever, cough, and fatigue, to severe, including difficulty breathing and complications like pneumonia. Vulnerable populations, such as
the elderly and those with underlying health conditions, faced increased risks of severe outcomes. The pandemic had profound impacts on
global health systems, economies, and daily life, prompting widespread lockdowns, travel restrictions, and shifts to remote work and
education. In response, scientists and pharmaceutical companies rapidly developed vaccines, which became a crucial tool in combating
the virus and reducing its spread. Despite progress in managing the pandemic, the emergence of new variants posed ongoing challenges,
underscoring the importance of public health measures, vaccination campaigns, and global cooperation to mitigate the virus's impacts.
'''

# Create structured messages for LLM
messages = [
    SystemMessage(content="You are an expert copywriter with expertise in summarizing documents."),
    HumanMessage(content=f"Please provide a short and concise summary of the following text:\n\nTEXT:\n{text}")
]

# Invoke the LLM to get the summary
output = llm.invoke(messages)

# Count tokens
input_tokens = llm.get_num_tokens(text)              # Tokens in input text
output_tokens = llm.get_num_tokens(output.content)  # Tokens in LLM output
total_tokens = input_tokens + output_tokens         # Total tokens

# Print results
print(f"Input Tokens : {input_tokens}")
print(f"Output Tokens: {output_tokens}")
print(f"Total Tokens : {total_tokens}")


### Import Modules for Summarization Chains
This cell demonstrates how to import the required classes for building a **document summarization pipeline**:  

- `load_summarize_chain` → Utility function to create an LLM chain specialized for summarizing documents.  
- `Document` → Class representing a document that can be processed by the chain.  


In [ ]:
# Import the summarization chain loader
from langchain.chains import load_summarize_chain

# Import the Document class
from langchain.docstore.document import Document



### Upload a Text File and Create a Document
This cell demonstrates how to:  
1. Upload a local text file to Colab.  
2. Read the file content.  
3. Create a `Document` object for use in LangChain summarization or other chains.  
4. Preview the first 500 characters of the text.
*italicized text*

In [ ]:

# Import files module from Colab
from google.colab import files
from langchain.docstore import document

# Upload a local file (interactive file chooser will appear)
uploaded = files.upload()

# Read the uploaded file content
with open('sj.txt', encoding='utf-8') as f:
    text = f.read()

# Create a LangChain Document object
docs = [document.Document(page_content=text)]

# Preview the first 500 characters of the text
print(text[:500])


Saving sj.txt to sj.txt
I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.

The first story is about connecting the dots.

I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit.


### Summarize Text Using a Custom Chat Prompt
This cell demonstrates how to:  
1. Define a concise summary prompt using `ChatPromptTemplate`.  
2. Load a summarization chain (`load_summarize_chain`) with the prompt.  
3. Run the chain on a `Document` object to get the summary.


In [ ]:
# Import ChatPromptTemplate if not already imported
from langchain.prompts import ChatPromptTemplate
from langchain.chains import load_summarize_chain

# Define a custom template for summarization
template = '''Write a concise and short summary of the following text.\nTEXT: {text}'''

# Create a ChatPromptTemplate from the template
prompt = ChatPromptTemplate.from_template(template)

# Load the summarization chain using the LLM and the custom prompt
chain = load_summarize_chain(
    llm,                  # Your AzureChatOpenAI LLM
    chain_type="stuff",   # "stuff" chain type combines all docs into one prompt
    prompt=prompt         # Use the custom prompt
)

# Run the summarization chain on the Document object
output = chain.run(docs)

# Print the summarized content
print(output)


In a commencement speech, Steve Jobs shares three impactful life lessons through personal stories. First, he emphasizes the importance of trusting life’s journey and connecting the dots in hindsight, recounting how dropping out of college allowed him to discover calligraphy, which later influenced the design of the Macintosh. Second, he reflects on love and loss, describing how getting fired from Apple led to a creative renaissance, founding NeXT and Pixar, and ultimately returning to Apple. Finally, he discusses the inevitability of death, urging the audience to live authentically, follow their hearts, and avoid living according to others' expectations. His closing message: "Stay Hungry. Stay Foolish."


### Split Large Text into Chunks for Summarization
This cell demonstrates how to handle **large documents** that exceed the token limit of your LLM (e.g., 10000 tokens).  

- `RecursiveCharacterTextSplitter` → Splits text into manageable chunks.  
- `chunk_size` → Maximum size of each chunk (here 10000 characters/tokens).  
- `chunk_overlap` → Overlap between chunks to preserve context.  
- Useful for **long documents** to avoid exceeding LLM token limits.


In [ ]:
# Import required classes
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],  # Split on paragraphs or lines
    chunk_size=10000,           # Max chunk size (characters/tokens)
    chunk_overlap=100           # Overlap to maintain context
)

# Split the text into chunks (each becomes a Document)
chunks = text_splitter.create_documents([text])

# Check how many chunks were created
len(chunks)


2

### Summarize Large Documents with Map-Reduce
This cell demonstrates how to summarize **large text split into chunks**:  

- `chain_type="map_reduce"` → First summarizes each chunk individually (**map step**) and then combines the summaries (**reduce step**) into a final summary.  
- Useful for documents exceeding the LLM token limit.  
- Input is a list of `Document` objects created by a text splitter.


In [ ]:
# Load the map-reduce summarization chain
chain = load_summarize_chain(
    llm,                # Your AzureChatOpenAI LLM
    chain_type="map_reduce"  # Summarize each chunk and then combine
)

# Run the summarization chain on the chunked documents
output = chain.run(chunks)

# Print the final summary
print(output)


Steve Jobs' speech shares three life lessons through personal stories: trusting that life’s events connect in hindsight, loving your work and persevering through setbacks, and embracing life’s fragility by living authentically and boldly. He urges graduates to trust their intuition, pursue passions, resist conformity, and stay curious and adventurous, embodying the mantra "Stay Hungry. Stay Foolish" as they face life's finite nature.


### Load PDF Documents
This cell demonstrates how to load a PDF file into LangChain for processing:  

- `PyPDFLoader` → Reads a PDF file and converts each page into a `Document` object.  
- Useful for **summarization, question answering, or any LLM tasks** on PDF content.

In [ ]:
# Import the PyPDFLoader
from langchain.document_loaders import PyPDFLoader



### Install PDF Processing Libraries
This cell installs Python packages required to work with PDFs in Colab:  

- `pdf2image` → Convert PDF pages to images.  
- `pdfminer` → Extract text from PDF files.  
- `pypdf` → Read and manipulate PDF files programmatically.


In [ ]:
# Install PDF processing libraries
!pip install -q pdf2image pdfminer pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 38.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.0 MB/s eta 0:00:00


### Upload and Load a PDF Document
This cell demonstrates how to:  
1. Upload a PDF file to Colab and automatically detects its filename.
2. Load it using `PyPDFLoader` to create `Document` objects for each page.  
3. The loaded documents can then be used in summarization or other LLM tasks.

In [ ]:
# Import required modules
from google.colab import files
from langchain.document_loaders import PyPDFLoader

# Upload PDF file (interactive file chooser)
uploaded = files.upload()

# Automatically get the uploaded filename
pdf_filename = list(uploaded.keys())[0]
print(f"Uploaded file detected: {pdf_filename}")

# Load the PDF using PyPDFLoader
pdf_loader = PyPDFLoader(pdf_filename)

# Extract documents (one Document per page)
data = pdf_loader.load()

# Preview the first page content
print(data[0].page_content[:500])


Saving attention_is_all_you_need.pdf to attention_is_all_you_need.pdf
Uploaded file detected: attention_is_all_you_need.pdf
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent o


### Print First Page of PDF
This cell prints the **entire content** of the first page of the loaded PDF document.


In [ ]:
# Print the full content of the first page
print(data[0].page_content)


Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring signiﬁcantly
less time to train. Our model 

### Check Number of Chunks
After splitting a large text or PDF into smaller `Document` chunks (e.g., using `RecursiveCharacterTextSplitter`),  
this cell shows how many chunks were created.


In [ ]:
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],  # Split on paragraphs or lines
    chunk_size=10000,           # Max chunk size (characters/tokens)
    chunk_overlap=100           # Overlap to maintain context
)

# Split the text into chunks (each becomes a Document)
chunks = text_splitter.split_documents(data)

# Check how many chunks were created
len(chunks)


15

### Summarize Documents Using Refine Chain
This cell demonstrates how to summarize **multiple document chunks** using the `refine` strategy:  

- `chain_type='refine'` → Iteratively refines the summary as each chunk is processed.  
- Useful for **large documents** where each chunk can update and improve the previous summary.  
- Input: list of `Document` objects (chunks).  
- Output: final refined summary of all chunks.


In [ ]:
# Load the refine summarization chain
chain = load_summarize_chain(
    llm=llm,           # Your AzureChatOpenAI LLM
    chain_type='refine', # Refine strategy to iteratively improve summary
    verbose=True        # Print intermediate steps
)

# Run the chain on the chunked documents
output_summary = chain.invoke(chunks)





> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine

### Print Refined Summary
This cell prints the **final summary** generated by the `refine` chain after processing all document chunks.


In [ ]:
# Print the final refined summary
print(output_summary)


{'input_documents': [Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2017-12-07T01:03:15+00:00', 'author': '', 'keywords': '', 'moddate': '2017-12-07T01:03:15+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention_is_all_you_need.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on com

### Custom Initial and Refine Summarization Prompts
This cell demonstrates how to create **custom prompt templates** for summarizing text using a **refine chain**:  

1. `initial_prompt` → Generates the first concise summary of the text.  
2. `refine_prompt` → Refines the initial summary with additional context, formatting the output as:
   - Introduction paragraph
   - Bullet points (if possible)
   - Conclusion phrase


In [ ]:
# Import PromptTemplate if not already imported
from langchain.prompts import PromptTemplate

# Initial summarization prompt
prompt_template = """Write a concise summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:"""

initial_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['text']
)

# Refine summarization prompt
refine_template = '''
Your job is to produce a final summary.
I have provided an existing summary up to a certain point: {existing_answer}.
Please refine the existing summary with some more context below.
------------
{text}
------------
Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
'''

refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text']
)

### Summarize Using Custom Initial & Refine Prompts
This cell demonstrates how to:  

1. Use `initial_prompt` for the first concise summary.  
2. Use `refine_prompt` to iteratively refine the summary with additional context.  
3. Run the `refine` chain on document chunks.  
4. Print only the final refined summary text.


In [ ]:
# Load the refine summarization chain with custom prompts
chain = load_summarize_chain(
    llm=llm,                        # AzureChatOpenAI instance
    chain_type='refine',             # Refine chain type
    question_prompt=initial_prompt,  # Initial summarization prompt
    refine_prompt=refine_prompt,     # Refine prompt template
    return_intermediate_steps=False  # Only return final summary
)

# Run the chain on the chunked documents
output_summary = chain.invoke(chunks)

# Display the final refined summary
print(output_summary['output_text'])


### Final Summary  

#### Introduction  
The Transformer architecture signifies a monumental shift in the evolution of machine learning and artificial intelligence. Introduced in the seminal paper **"Attention Is All You Need"**, it revolutionized sequence modeling with the introduction of the **self-attention mechanism**, allowing for direct computation of relationships between elements in a sequence without relying on the sequential processes inherent in traditional models such as recurrent neural networks (RNNs) or convolutional neural networks (CNNs). By overcoming bottlenecks like vanishing gradients and inefficient scaling, the Transformer has become a cornerstone of modern AI systems, enabling breakthroughs across domains spanning **natural language processing (NLP)**, **computer vision**, **multi-modal frameworks**, **time-series predictions**, and beyond. Models like **BERT**, **GPT**, **T5**, and **Vision Transformers (ViT)** have further amplified its transformative power in